In [1]:
import face_recognition
import cv2
import numpy as np
from matplotlib import pyplot as plt
import glob
from imutils import paths
from pathlib import Path
import pandas as pd
import datetime
import time
from openpyxl import load_workbook

In [2]:
imdir = 'Photos/'
ext = ['png','jpg','gif']
files =[]

[files.extend(glob.glob(imdir+'*.'+e)) for e in ext]
names= [Path(path).stem for path in files]
images = [cv2.imread(file) for file in files]

def res(x):
    if x[:,:,0].shape!=(1137,853):
        return cv2.resize(x, (853,1137))
    else:
        return x
images_re = [res(img) for img in images]
[cv2.imwrite(files[i], images_re[i]) for i in range(len(files)) if images[i][:,:,0].shape!=(1137,853)]
face_reco = [face_recognition.load_image_file(x) for x in files]
face_enc = [face_recognition.face_encodings(x)[0] for x in face_reco]

In [8]:

known_face_encodings = face_enc
known_face_names = names

# Initialize some variables
face_locations = []
face_encodings = []

process_this_frame = True
flag =[0]*len(names)
Time_stamp = {'Date':[],'Name':[],'Time':[]}
video_capture = cv2.VideoCapture('http://192.168.1.104:81/stream')
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding,tolerance=0.45)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)
            if flag[best_match_index]==0:
                n = time.time()
                Time_stamp['Name'].append(name)
                Time_stamp['Date'].append(datetime.datetime.now().date())
                Time_stamp['Time'].append(datetime.datetime.now().time())
                flag[best_match_index]=1
            elif flag[best_match_index]==1 and time.time()-n>2:
                n = time.time()
                Time_stamp['Name'].append(name)
                Time_stamp['Date'].append(datetime.datetime.now().date())
                Time_stamp['Time'].append(datetime.datetime.now().time())


    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 6)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if (cv2.waitKey(1) & 0xFF == ord('q')) or (cv2.waitKey(1) & datetime.datetime.now().time().hour>20):#0xFF == ord('q')or 
        break

# First and last entry
try:
    df = pd.DataFrame(data=Time_stamp)
    df.set_index('Date',inplace=True)
    final_df = pd.DataFrame()
    df = df.loc[datetime.datetime.now().date()]
    for name in names:
        df1 = df[df['Name']==name]
        final_df= pd.concat([final_df,df1.head(1),df1.tail(1)])

    #Excel append
    book = load_workbook('Employees.xlsx')
    writer = pd.ExcelWriter('Employees.xlsx', engine='openpyxl')
    writer.book = book
    writer.sheets = {ws.title: ws for ws in book.worksheets}
    for sheetname in writer.sheets:
        final_df.to_excel(writer,sheet_name='Sheet1', startrow=writer.sheets['Sheet1'].max_row, index = True,header= False)

    writer.save()
except:
    video_capture.release()
    cv2.destroyAllWindows()
video_capture.release()
cv2.destroyAllWindows()    